In [ ]:
!pip install geemap

import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=DqhoiUt2Ac2rCtkCww0Igs23NKfU70sgfYq8puMpJXw&tc=gsGu7VcX0jOZ3iFQl8p_U8c_pJ57cmXpxEJdVy0Fd_4&cc=-QHRDxVOKaDpCBPxdxWWAVYdLkqjftHUtP4whHuNgJE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BWsV2Cy3EKnXvCFut5aDKGDga4eQMN_s5roYMMUVFIGCZyZdm3DIQU

Successfully saved authorization token.


In [ ]:
Map = geemap.Map()

In [ ]:
NDVI_BAND = "NDVI"
LEAF_BANDS = ["Fpar", "Lai"]
SOIL_MOI_BAND = "SoilMoi00_10cm_tavg"
ERA5_BANDS = ["total_precipitation", "u_component_of_wind_10m", "v_component_of_wind_10m"]
RESPONSE = "T21"
BANDS = [NDVI_BAND] + LEAF_BANDS + [SOIL_MOI_BAND] + ERA5_BANDS

In [ ]:
ndvi = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').filterDate('2019-07-01', '2019-07-14').select(NDVI_BAND).median()
leaf = ee.ImageCollection('MODIS/061/MCD15A3H').filterDate('2019-07-01', '2019-07-14').select(LEAF_BANDS).median()
soil = ee.ImageCollection('NASA/FLDAS/NOAH01/C/GL/M/V001').filterDate('2019-07-01', '2019-07-14').select(SOIL_MOI_BAND).median()
era5 = ee.ImageCollection('ECMWF/ERA5/MONTHLY').filterDate('2019-07-01', '2019-07-14').select(ERA5_BANDS).median()
firms = ee.ImageCollection('FIRMS').filterDate('2019-07-15', '2019-07-28').select(RESPONSE).max().unmask(0).divide(510)

In [ ]:
featureStack = ee.Image.cat([ndvi, leaf, soil, era5]).float()

In [ ]:
has_fire = firms.ceil().byte().rename('HAS_FIRE')

In [ ]:
# trainingPolys = ee.FeatureCollection('projects/google/DemoTrainingGeometries')
# evalPolys = ee.FeatureCollection('projects/google/DemoEvalGeometries')
# trainingPolys = trainingPolys.merge(evalPolys)

In [ ]:
# caBoundary = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_CODE', 46))

In [ ]:
# caBoundary = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', 'US'))

In [ ]:
caBoundary = ee.FeatureCollection('projects/ee-yzyly1992/assets/CA_Boundary')

In [ ]:
samples = has_fire.stratifiedSample(
    2000,
    classBand = "HAS_FIRE",
    region = caBoundary.geometry(),
    scale = 1000,
    geometries = True
)

In [ ]:
print(samples.size().getInfo())

4000


In [ ]:
# Map.addLayer(samples, {}, 'samples', False)

In [ ]:
allData = featureStack.sampleRegions(
    collection = samples,
    properties = ['HAS_FIRE'],
    scale = 30,
    geometries = False
).randomColumn()

In [ ]:
split = 0.7
training = allData.filter(ee.Filter.lt('random', split))
validation = allData.filter(ee.Filter.gte('random', split))

In [ ]:
train_task = ee.batch.Export.table.toDrive(
  collection = training,
  folder = 'GEE',
  description = "export training data - wildf",
  fileNamePrefix = 'wildfire_train_4000_',
  fileFormat = 'CSV',
  selectors = BANDS + ['HAS_FIRE']
)

train_task.start()

In [ ]:
val_task = ee.batch.Export.table.toDrive(
  collection = validation,
  folder = 'GEE',
  description = "export val data - wildf",
  fileNamePrefix = 'wildfire_val_4000_',
  fileFormat = 'CSV',
  selectors = BANDS + ['HAS_FIRE']
)

val_task.start()

In [ ]:
classifier = ee.Classifier.smileCart().train(training, 'HAS_FIRE', BANDS)

In [ ]:
classified = featureStack.select(BANDS).classify(classifier)

In [ ]:
Map.addLayer(classified,
             {'min': 0, 'max': 1, 'bands': ['classification'], 'palette':['green', 'red']},
             'wildfire',False)

In [ ]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())
print('F1 score: ', trainAccuracy.fscore().getInfo())

Resubstitution error matrix:  [[1221, 0], [0, 1320]]
Training overall accuracy:  1
F1 score:  [1, 1]


In [ ]:
validated = validation.classify(classifier);

In [ ]:
testAccuracy = validated.errorMatrix('HAS_FIRE', 'classification')
print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())
print('F1 score: ', testAccuracy.fscore().getInfo())

Validation error matrix:  [[416, 59], [56, 506]]
Validation overall accuracy:  0.8891031822565092
F1 score:  [0.8785638859556494, 0.8979591836734694]
